In [1]:
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
import os
import json


In [2]:
%pwd

'c:\\Users\\Admin\\Desktop\\table detection\\TRUST'

In [3]:
path = "\\PubTabNet\\examples\\PubTabNet_Examples.jsonl"
cwd = os.getcwd()
anno_list = []
with open(cwd+path) as f:
    for object in f:
        anno = json.loads(object)
        anno_list.append(anno)
print(anno_list[0]['imgid'])

0


In [4]:
def get_union(a, m):
    b = []
    for begin,end in sorted(a):
        if b and b[-1][1] >= begin - 1:
            b[-1][1] = max(b[-1][1], end)
        else:
            b.append([begin, end])
    prev = 0
    c = []
    for begin,end in b:
        c.append([prev, begin])
        prev = end
    c.append([prev, m])
    return c

N = 100 #max number of rows
M = 100 #max number of columns
img_cnt = 0
json_dicts = []
trust_N, trust_M = 100, 100
for img_anno in anno_list:
    n, m = 100, 100
    img = cv2.imread('C:\\Users\\Admin\\Desktop\\table detection\\TRUST\\PubTabNet\\examples\\'+img_anno['filename'])
    h, w, c = img.shape
    col_ranges = []
    row_ranges = []
    col_separator_ranges = []
    row_separator_ranges = []
    bboxes = []
    for cell in img_anno['html']['cells']:
        if 'bbox' in cell:
            coords = cell['bbox']
            # col_ranges.append((coords[0], coords[2]))
            # row_ranges.append((coords[1], coords[3]))
            bboxes.append(coords) #add angle to bounding box
        else:
            bboxes.append([])
    cell_matrix = [[-1]*M for _ in range(N)] #we have to find this matrix
    vertex_merge = [[[0]*4 for _ in range(M)] for _ in range(N)]
    cell_spanning_detail = [] #(sr, er, sc, ec), we need cell spanning information
    token_list = img_anno['html']['structure']['tokens']
    i = 0
    cell_cnt = 0
    row_cnt = 0
    # max_row = 0
    # max_col = 0
    while i < len(token_list):
        if token_list[i]=='<tr>':
            # print(row_cnt)
            i += 1
            col_cnt = 0
            while cell_matrix[row_cnt][col_cnt] > -1:
                col_cnt += 1
            while token_list[i] != '</tr>':
                token = token_list[i]
                if token == '<td>':
                    cell_spanning_detail.append([row_cnt, row_cnt, col_cnt, col_cnt])
                    cell_matrix[row_cnt][col_cnt] = cell_cnt
                    i += 2
                    col_cnt += 1
                    cell_cnt += 1
                else:
                    i += 1
                    token = token_list[i]
                    rowspan = 1
                    colspan = 1
                    if 'rowspan' in token:
                        ind = token.find("\"")
                        ind += 1
                        ind2 = token[ind:].find("\"")
                        rowspan = int(token[ind:ind+ind2])
                    if 'colspan' in token:
                        ind = token.find("\"")
                        ind += 1
                        ind2 = token[ind:].find("\"")
                        colspan = int(token[ind:ind+ind2])
                    for j in range(rowspan):
                        for k in range(colspan):
                            cell_matrix[row_cnt+j][col_cnt+k] = cell_cnt
                    cell_spanning_detail.append([row_cnt, row_cnt+rowspan-1, col_cnt, col_cnt+colspan-1])
                    col_cnt += colspan
                    i += 3
                    cell_cnt += 1
            if row_cnt==0:
                m = col_cnt
            # print(cell_matrix[row_cnt])
            row_cnt += 1
        i += 1
    n = row_cnt
    print(f"image {img_anno['filename']} has {n} rows, {m} columns.")
    # print(cell_matrix)
    # print(img_anno['filename'])
    # if img_anno['filename']=="PMC4003957_018_00.png":
        # for k in range(30):
        #     print(cell_matrix[k])
    for i in range(n):
        for j in range(m):
            # if cell_matrix[i][j] != -1:
            cell_cnt = cell_matrix[i][j]
            if len(bboxes[cell_cnt])==0:
                continue
            if cell_spanning_detail[cell_cnt][1]-cell_spanning_detail[cell_cnt][0]==0:
                row_ranges.append((bboxes[cell_cnt][1], bboxes[cell_cnt][3]))
            if cell_spanning_detail[cell_cnt][3]-cell_spanning_detail[cell_cnt][2]==0:
                col_ranges.append((bboxes[cell_cnt][0], bboxes[cell_cnt][2]))
    col_separator_ranges = get_union(col_ranges, w-1)
    row_separator_ranges = get_union(row_ranges, h-1)
    

    for i in range(n+1):
        for j in range(m+1):
            pairs = [((i-1,j-1),(i-1,j)), ((i, j-1), (i,j)), ((i-1,j-1),(i,j-1)), ((i-1,j),(i,j))]
            for k in range(4):
                pair = pairs[k]
                a, b, c, d = pair[0][0], pair[1][0], pair[0][1], pair[1][1]
                if (a < 0 or a > N-1 
                    or b < 0 or b > N-1
                    or c < 0 or c > M-1
                    or d < 0 or d > M-1):
                    vertex_merge[i][j][k] = 0
                else:
                    vertex_merge[i][j][k] = int(cell_matrix[a][c]==cell_matrix[b][d])
    # if img_anno['filename']=="PMC4003957_018_00.png":
    #     for i in range(20):
    #         for j in range(20):
    #             print(vertex_merge[i][j])

    json_dicts.append({
        'filename': img_anno['filename'],
        'col_separators':col_separator_ranges,
        'row_separators':row_separator_ranges,
        'bboxes': bboxes,
        # 'cell_spanning': cell_spanning_detail,
        'vertex_merge': vertex_merge,
        'angle': 0
    })
    
with open("sample1.json", "w") as outfile:
    json.dump(json_dicts, outfile)
    




image PMC4840965_004_00.png has 28 rows, 4 columns.
image PMC4517499_004_00.png has 4 rows, 7 columns.
image PMC4776821_005_00.png has 5 rows, 5 columns.
image PMC1626454_002_00.png has 9 rows, 12 columns.
image PMC2838834_005_00.png has 36 rows, 7 columns.
image PMC5897438_004_00.png has 11 rows, 2 columns.
image PMC3907710_006_00.png has 4 rows, 5 columns.
image PMC3519711_003_00.png has 11 rows, 4 columns.
image PMC5198506_004_00.png has 7 rows, 3 columns.
image PMC5679144_002_01.png has 11 rows, 2 columns.
image PMC5134617_013_00.png has 9 rows, 8 columns.
image PMC2753619_002_00.png has 2 rows, 6 columns.
image PMC3826085_003_00.png has 18 rows, 5 columns.
image PMC5577841_001_00.png has 5 rows, 4 columns.
image PMC2759935_007_01.png has 14 rows, 9 columns.
image PMC4003957_018_00.png has 21 rows, 4 columns.
image PMC4682394_003_00.png has 13 rows, 8 columns.
image PMC4172848_007_00.png has 18 rows, 7 columns.
image PMC5332562_005_00.png has 31 rows, 4 columns.
image PMC5402779_00